In [8]:
import csv
import os

import azure.identity
import openai
from dotenv import load_dotenv
from lunr import lunr

In [13]:
load_dotenv()
API_HOST = os.getenv("API_HOST", "github")
 
if API_HOST == "github":
    client = openai.OpenAI(base_url="https://models.github.ai/inference", api_key=os.environ["GITHUB_TOKEN"])
    MODEL_NAME = os.getenv("GITHUB_MODEL", "openai/gpt-4o")

In [12]:
# Index the data from the CSV
with open("hybrid.csv") as file:
    reader = csv.reader(file)
    rows = list(reader)
documents = [{"id": (i + 1), "body": " ".join(row)} for i, row in enumerate(rows[1:])]
index = lunr(ref="id", fields=["body"], documents=documents)

# Get the user question
user_question = "how fast is the prius v?"

# Search the index for the user question
results = index.search(user_question)
matching_rows = [rows[int(result["ref"])] for result in results]

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matching_rows)

print("Found matches:")
print(matches_table)

# Now we can use the matches to generate a response
SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any info that is not in the provided sources.
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.3,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": f"{user_question}\nSources: {matches_table}"},
    ],
)

print(f"\nResponse from {API_HOST}: \n")
print(response.choices[0].message.content)

Found matches:
vehicle | year | msrp | acceleration | mpg | class
 ---  |  ---  |  ---  |  ---  |  ---  |  --- 
Prius | 2004 | 20355.64 | 9.9 | 46.0 | Midsize
Prius | 2013 | 24200.0 | 10.2 | 50.0 | Midsize
Prius V | 2011 | 27272.28 | 9.51 | 32.93 | Midsize
Prius C | 2012 | 19006.62 | 9.35 | 50.0 | Compact
Prius PHV | 2012 | 32095.61 | 8.82 | 50.0 | Midsize
Prius C | 2013 | 19080.0 | 8.7 | 50.0 | Compact
Prius Plug-in | 2013 | 32000.0 | 9.17 | 50.0 | Midsize
Prius (1st Gen) | 1997 | 24509.74 | 7.46 | 41.26 | Compact
Prius (2nd Gen) | 2000 | 26832.25 | 7.97 | 45.23 | Compact
Prius (3rd Gen) | 2009 | 24641.18 | 9.6 | 47.98 | Compact
Prius alpha (V) | 2011 | 30588.35 | 10.0 | 72.92 | Midsize

Response from github: 

The Prius V from 2011 has an acceleration time of **9.51 seconds**. This refers to the time it takes to go from 0 to 60 mph.
